pydamicOutputparser

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [4]:
email_conversation = email_conversation = """From: 박지영 (jiyoung.park@techsolutions.kr)
To: 김민수 (minsu@globaltech.com)
Subject: AI 솔루션 도입 제안 및 컨설팅 미팅 요청

김민수 부장님께,

안녕하세요, 테크솔루션즈의 박지영 이사입니다. 귀사의 디지털 전환 계획에 대해 알게 되어 연락드립니다. 저희 테크솔루션즈는 AI 기반 비즈니스 솔루션 분야에서 10년 이상의 경험을 보유한 선도기업입니다.

귀사의 비즈니스 프로세스를 최적화할 수 있는 맞춤형 AI 솔루션을 제안드리고 싶습니다. 특히 데이터 분석, 자동화 시스템, 예측 모델링 분야에서 혁신적인 솔루션을 보유하고 있습니다.

구체적인 논의를 위해 다음 달 3월 20일 오후 2시에 미팅을 제안드립니다. 귀사의 요구사항과 목표를 자세히 파악하고, 최적의 솔루션을 제시해드리고 싶습니다.

검토 부탁드립니다.

감사합니다.

박지영
이사
테크솔루션즈
"""

출력 파서를 사용하지 않는 경우

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출하세요. \n {email_conversation}"
)

chain = prompt | llm | StrOutputParser()

answer = chain.invoke({"email_conversation" : email_conversation})

print(answer)

- 발신자: 박지영, 테크솔루션즈 이사
- 수신자: 김민수, 글로벌테크 부장
- 주제: AI 솔루션 도입 제안 및 컨설팅 미팅 요청
- 주요 내용:
  - 테크솔루션즈는 AI 기반 비즈니스 솔루션 분야에서 10년 이상의 경험을 보유한 선도기업임.
  - 데이터 분석, 자동화 시스템, 예측 모델링 분야에서 맞춤형 AI 솔루션 제안.
  - 3월 20일 오후 2시에 미팅 제안.
  - 귀사의 요구사항과 목표를 파악하여 최적의 솔루션 제시 희망.


In [6]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="본문 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [7]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "본문 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [8]:
prompt = PromptTemplate.from_template(
"""
You are helpful assistant. Please answer the following question in korean

Question:
{question}

Email CONVERSATION:
{email_conversation}

FORMAT:
{format}

"""
)

prompt = prompt.partial(format=parser.get_format_instructions())

In [9]:
chain = prompt | llm

response = chain.invoke(
    {
        "email_conversation" : email_conversation,
        "question": "이메일 내용 중 중요한 내용을 추출해주세요."
    }
)

print(response.content)

```json
{
    "person": "박지영",
    "email": "jiyoung.park@techsolutions.kr",
    "subject": "AI 솔루션 도입 제안 및 컨설팅 미팅 요청",
    "summary": "테크솔루션즈의 박지영 이사가 김민수 부장님에게 AI 기반 비즈니스 솔루션을 제안하며, 데이터 분석, 자동화 시스템, 예측 모델링 분야에서 혁신적인 솔루션을 제공할 수 있음을 강조하고, 3월 20일 오후 2시에 미팅을 제안함.",
    "date": "3월 20일 오후 2시"
}
```


In [10]:
structured_output = parser.parse(response.content)

print(structured_output.person)
print(structured_output.email)
print(structured_output.subject)
print(structured_output.summary)

박지영
jiyoung.park@techsolutions.kr
AI 솔루션 도입 제안 및 컨설팅 미팅 요청
테크솔루션즈의 박지영 이사가 김민수 부장님에게 AI 기반 비즈니스 솔루션을 제안하며, 데이터 분석, 자동화 시스템, 예측 모델링 분야에서 혁신적인 솔루션을 제공할 수 있음을 강조하고, 3월 20일 오후 2시에 미팅을 제안함.


In [11]:
chain = prompt | llm | parser

response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용 중 중요한 내용을 추출해 주세요"
    }
)

print(response)

person='박지영' email='jiyoung.park@techsolutions.kr' subject='AI 솔루션 도입 제안 및 컨설팅 미팅 요청' summary='테크솔루션즈의 박지영 이사가 김민수 부장에게 AI 기반 비즈니스 솔루션을 제안하며, 데이터 분석, 자동화 시스템, 예측 모델링 분야에서의 혁신적인 솔루션을 소개하고자 한다. 3월 20일 오후 2시에 미팅을 제안하며, 귀사의 요구사항과 목표를 파악하여 최적의 솔루션을 제시하고자 한다.' date='3월 20일 오후 2시'


In [12]:
llm_with_structured = ChatOpenAI(
    temperature=0,
    model="gpt-4o"
).with_structured_output(EmailSummary)

In [13]:
answer = llm_with_structured.invoke(email_conversation)

answer

EmailSummary(person='박지영', email='jiyoung.park@techsolutions.kr', subject='AI 솔루션 도입 제안 및 컨설팅 미팅 요청', summary='테크솔루션즈의 박지영 이사가 김민수 부장님께 AI 기반 비즈니스 솔루션 도입을 제안하며, 데이터 분석, 자동화 시스템, 예측 모델링 분야의 혁신적인 솔루션을 소개하고자 합니다. 이를 위해 3월 20일 오후 2시에 미팅을 제안드립니다.', date='3월 20일 오후 2시')